In [2]:
using Revise

# A tutorial for the package ```ChebyshevFiltering```.

In this notebook we will provide a tutorial on how to use the the package ```ChebyshevFiltering``` for sparse diagonalization of Hermitian matrices (both real and complex).

Let us first load the package

In [3]:
using ChebyshevFiltering
include("/home/dario/Dropbox/delocalization_and_growth/revision_chaos_paper_code/QMB_module.jl")
using .ImpurityUtilities

and now let's create an SYK-like Hamiltonian. We also renormalize its spectrum to lie in the interval $\left[-1 , \, 1 \right]$.

In [4]:
const number_of_majorana = 20
majorana_matrices = majorana_matrices_chiral(number_of_majorana)
hamiltonian_matrix, edges_graph, weights_graph = hamiltonian_chiral(number_of_majorana, majorana_matrices, 4, 1.0)
ChebyshevFiltering.renormalization_hamiltonian!(hamiltonian_matrix)

512×512 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 14848 stored entries:
⎡⢿⣷⣯⣮⡿⠅⠀⢸⣾⡿⠇⠀⠀⠀⠀⠳⣄⠘⣶⡟⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠈⠳⣄⣝⣾⡮⎤
⎢⡫⣿⣿⣿⣼⣢⣶⣾⠛⠙⠂⠀⠀⠀⠀⢦⣼⠿⣏⠙⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⢐⢦⣮⡻⣗⠙⎥
⎢⠟⠏⠲⣻⣿⣿⣟⣦⡠⣤⣄⠀⢠⣀⣤⡾⠛⠆⠈⠳⠀⠀⠀⠈⠳⣄⠀⠀⠀⠠⣄⠀⢤⣨⡻⣿⠝⠆⠈⠳⎥
⎢⣀⣀⣸⣿⠻⣽⣿⣿⣾⢫⡉⢳⣤⡟⢧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⢀⡈⣳⣷⣟⢯⠈⠳⣄⠀⠀⎥
⎢⣾⡿⣟⠀⠀⣮⡾⣛⣿⣿⣾⢿⡉⠓⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣔⣿⣿⡋⠳⣄⠀⠀⠈⠳⣄⎥
⎢⠉⠁⠈⠀⠀⠙⢧⣈⣾⣟⢿⣷⣯⣪⡿⠅⠀⢸⣾⡿⠳⣄⠀⠀⠈⠳⣄⢽⣮⡻⠳⣅⠁⠀⠈⠀⠀⠀⠀⠈⎥
⎢⠀⠀⠀⠀⠀⢲⣤⠿⢧⠈⡫⣻⣿⣿⣼⣦⣶⡞⠛⠙⠀⠈⠳⣄⢐⣦⡺⡿⣝⠙⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⢤⡀⠠⣄⣠⡿⠉⠳⠀⠙⠟⠏⠲⣿⣿⣿⣏⡦⡠⣤⢄⠀⢠⡨⣷⣟⠟⠆⠈⠳⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⎥
⎢⣀⠙⣶⡟⠻⠄⠀⠀⠀⠀⣀⣀⣸⠿⠫⡽⣿⣿⣿⢯⡈⣱⣼⠿⠮⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⎥
⎢⣼⠿⣏⠙⢦⡀⠀⠀⠀⠀⣾⡿⣟⠀⠀⣮⡿⣟⢿⣷⣾⡿⡋⠱⢄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⎥
⎢⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠑⢆⣨⣾⡿⢿⣷⣽⣾⡻⠀⠀⣽⣾⡿⠀⠀⠀⠀⠈⠳⣄⣹⣶⡟⎥
⎢⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⡲⣶⡟⢏⡈⣳⣿⣿⣿⣞⣢⣶⡏⠉⠉⠀⠀⠀⠀⠐⣦⣼⠿⣄⠉⎥
⎢⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⢦⡀⠰⣴⣽⢿⡊⠃⠀⠑⠛⠊⠺⣹⣿⣿⣿⠦⣰⣴⣄⠀⢦⣀⣾⠋⠙⠂⠈⠓⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⣄⣝⣾⡮⠻⠅⠙⢦⡀⠀⣄⣤⡼⠿⠻⡟⣿⣿⣯⣪⡀⢳⣶⠛⠧⠀⠀⠀⠀⠀⎥
⎢⡀⠀⠀⠀⠀⡀⠀⢀⢙⢦⣮⡻⣗⠙⢦⡀⠀⠀⠙⢦⣾⡿⡇⠀⢐⣾⡫⣻⢿⣷⣽⡿⡉⢳⣄⠀⠀⡀⢀⣀⎥
⎢⠙⢦⡀⠀⠀⠙⢦⣨⣿⣿⠝⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢤⣈⣷⡿⣿⣿⣭⡾⡻⠀⠀⣽⣾⡿⎥
⎢⠀⠀⠙⢦⡀⣳⣽⢿⢯⡈⠁⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢳⣼⠛⢧⣈⣣⡿⣿⣿⣟⣦⣿⡏⠉⠉⎥
⎢⢦⡀⠰⣔⣿⣮⡋⠓⠀⠙⠂⠀⠀⠀⠙⢦⡀⠀⠀⠀⢦⡀⠰⣤⡾⠛⠉⠃⠀⠙⠛⠊⠻⣽⣿⣿⣯⠦⣰⣴⎥
⎢⣄⢽⣮⡻⠳⠅⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⣄⣹⣶⡟⠳⠀⠀⠀⠀⠠⣄⣤⡿⠿⠫⡟⣿⣿⣿⣪⎥
⎣⡺⡿⣝⠙⢦⡀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⣼⠿⡄⠙⢦⠀⠀⠀⠀⢰⣾⡿⡇⠀⢐⣾⡻⣻⢿⣷⎦

## The estimate of the density of states via Kernel Polynomial Method.

We now test how works the estimate of the density of states, computed by means of the Kernel Polynomial Method.

We start by fixing the relevant parameters, and the window of the spectrum where we want to estimate the density

In [5]:
max_degree_KPM = 150
stochastic_dimension_KPM = 30
lambda_min = -0.1
lambda_max = 0.1
search_target_ratio = 3 
e_max = 1.
e_min = -1. 
N_0 = 6.23
epsilon_convergence = 10^(-7) 
log_path = "/home/dario/Dropbox/delocalization_and_growth_data/log_try/"
log_file_name = "log_try.txt"

"log_try.txt"

and now we can compute the approximated density and compute the number of expected states

In [6]:
approximated_density = KPM_density(hamiltonian_matrix, max_degree_KPM, stochastic_dimension_KPM)
x_values = lambda_min:((lambda_max - lambda_min) / 200):lambda_max
y_values = [approximated_density(x) for x in x_values]

using NumericalIntegration
expected_value = NumericalIntegration.integrate(x_values, y_values)

98.26430615829746

which we compare against the actual value:

In [7]:
using LinearAlgebra
real_number = length([value  for value in eigvals(Matrix(hamiltonian_matrix)) if lambda_min < value < lambda_max])

98

We see that the agreement is excellent.

Now, we proceed by estimating the optimal parameters, *i.e.* the optimal number of search vectors and the optimal degree of the Chebyshev polynomials that should be used in the filtering procedure.

In [8]:
search_vector_numbers, polynomial_degree_optim = ChebyshevFiltering.optimal_search_degree(hamiltonian_matrix, lambda_min, lambda_max, search_target_ratio, max_degree_KPM, stochastic_dimension_KPM, e_max, e_min, N_0)

println(search_vector_numbers)
println(polynomial_degree_optim)

303
51


It works well.

In [9]:
search_vector_numbers = Int64(ceil(search_vector_numbers/Threads.nthreads()) * Threads.nthreads())
chebyshev_coeff = ChebyshevFiltering.expansion_coeff(polynomial_degree_optim, e_max, e_min, lambda_max, lambda_min)
damping_coeff = ChebyshevFiltering.Lanczos_coeff(polynomial_degree_optim, 2)
full_coeff = Array{Float64}(undef, length(damping_coeff))
@inbounds for i in eachindex(full_coeff) 
full_coeff[i] = chebyshev_coeff[i] * damping_coeff[i]
end
converged_target_values, converged_target_vectors = ChebyshevFiltering.polynomial_filtering(search_vector_numbers, polynomial_degree_optim, full_coeff, hamiltonian_matrix, lambda_max, lambda_min, epsilon_convergence, log_path, log_file_name)


([-0.09770173506163855, -0.09473442993968702, -0.09410529078044606, -0.09216795862778164, -0.08993481548531783, -0.08922282668484048, -0.0866282722683245, -0.08622525783796539, -0.08348880825398948, -0.08251401859476123  …  0.08047291015117793, 0.08345455544786609, 0.08580396455348419, 0.08714726798631424, 0.08780803739927177, 0.08960458738719417, 0.09302004666652522, 0.09395727257523927, 0.09579847238342437, 0.09711913084746152], Vector{ComplexF64}[[-0.027159825286316754 - 0.0248026952220574im, 0.027904867194128036 + 0.0063144916700504155im, 0.010943046271823586 - 0.045082365399239106im, -0.04784528300268136 - 0.0033971116119390415im, 0.0018767272933331348 + 0.06968054820829103im, -0.0395508234887984 + 0.020048766942696764im, -0.0036444140701777593 + 0.003535851191597775im, 0.032733646626514894 - 0.03267926128780392im, -0.05760174969209959 + 0.014477980836028276im, -0.03396323928277329 - 0.023656960169084407im  …  0.04040328750571095 - 0.03966064885089225im, 0.01090740311393933 + 0.00

In [11]:
isapprox(converged_target_values, [value  for value in eigvals(Matrix(hamiltonian_matrix)) if lambda_min < value < lambda_max])

true